## Dynamic Programming Algorithm (Final Version from Prajjwal Gandharv with all details before leaving the team)

Varibles which will have initial values at start of the algorithm (They will have to be derived from the data)-:
mu
pi
L
w


Possible sources to find the initial value of pi- Din00; also look at xbilat00



A dot and kappa dot are constant at 1 from 2007

In [1]:
nl = 200 # time periods
n = 50 # locations
j = 23 # industries+non-employment
i = n # locations = n for 3 to 4 dim variables
k = j # industries = j for 3 to 4 dim variables

23

In [2]:
μ = Array{Float64}(undef, n, j, i, k, nl+1) # migration shares
π_0 = Array{Float64}(undef, n, j, i, j, nl+1) # cross-expenditures
β = 0.99
ν = 1.1

1.1

# Initializing the data for the algorithm

### Reading in the Data for Mu

The mu.mat file contains mu values for migrations between 23 sectors and 50 states in an 1150 by 1150 matrix for each time period. More on the time periods-:


These time periods are quarters from 2000 to 2007. 7 years= 28 quarters. Hence there are 28 1150 by 1150 matrices. The last one of these 28 is supposed to be the starting value of mu for the dynamic programming algorithm. We will tranform each 1150 by 1150 matrix into a 50 x 23 x 50 x 23 matrix for easier indexing among different states and sectors.

In [3]:
using MAT
mudata = matread("mu.mat") # data for mu0 is mudata["series_mu_adj"][:, :, 28]
# use all the period mu's to calculate L0

Dict{String, Any} with 2 entries:
  "series_mu_adj" => [0.89191 0.000409915 … 0.0 0.0; 0.0151858 0.886235 … 0.0 0…
  "L0_initial"    => [0.00535588; 0.000192335; … ; 7.41149e-5; 0.000188085;;]

In [4]:
size(mudata["series_mu_adj"])

(1150, 1150, 28)

In [5]:
all_mus = mudata["series_mu_adj"]

1150×1150×28 Array{Float64, 3}:
[:, :, 1] =
 0.89191    0.000409915  0.00233679  …  8.17792e-7  0.0         0.0
 0.0151858  0.886235     0.0            0.0         0.0         0.0
 0.0662532  0.00685947   0.893431       0.0         0.0         0.0
 0.0421921  0.0          0.0            0.0         0.0         0.0
 0.0        0.0          0.0            0.0         0.0         0.0
 0.0        0.0173049    0.0103574   …  4.11926e-6  0.0         0.0
 0.0        0.0          0.027987       0.0         0.0         0.0
 0.0        0.0          0.0            0.0         0.0         0.0
 0.0394941  0.00769432   0.0            0.0         0.0         0.0
 0.0698735  0.0          0.0            0.0         0.0         0.0
 0.0198944  0.0          0.0         …  0.0         0.0         0.0
 0.0        0.0          0.0            0.0         0.0         0.0
 0.0210562  0.0          0.0            0.0         0.0         0.0
 ⋮                                   ⋱                          
 0.0   

In [6]:
# Function to convert 1150x1150 to 50x23x50x23 matrix
function convert_large_matrix(matrix_1150x1150)
    # Initialize the 50x23x50x23 matrix with zeros
    matrix_50x23x50x23 = zeros(Float64, 50, 23, 50, 23)
    
    for i in 1:1150
        for j in 1:1150
            # Calculate indices for the 50x23x50x23 matrix
            # Determine source location and sector
            src_location = div((i-1), 23) + 1
            src_sector = mod((i-1), 23) + 1
            # Determine destination location and sector
            dest_location = div((j-1), 23) + 1
            dest_sector = mod((j-1), 23) + 1

            # Assign values from 1150x1150 to 50x23x50x23 matrix
            matrix_50x23x50x23[src_location, src_sector, dest_location, dest_sector] = matrix_1150x1150[i, j]
        end
    end

    return matrix_50x23x50x23
end


convert_large_matrix (generic function with 1 method)

In [7]:
all_mus_reshaped = Array{Float64}(undef, n, j, i , k, 28)

50×23×50×23×28 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0

In [8]:
for a in 1:28
    all_mus_reshaped[:, :, :, :, a] = convert_large_matrix(all_mus[:, :, a])
end

In [9]:
all_mus_reshaped

50×23×50×23×28 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 0.89191     0.0151858   0.0662532   …  0.00610247  0.0364077   0.0196429
 0.0         0.0         0.0            0.0         0.0         0.0
 5.5014e-6   2.16834e-7  4.79669e-6     3.31442e-8  1.60549e-7  8.85311e-8
 6.03377e-6  3.24456e-8  1.32582e-6     3.34177e-8  2.37554e-7  1.4971e-7
 1.35629e-6  2.19503e-8  1.8806e-7      8.03099e-9  4.00535e-8  1.92235e-8
 7.72114e-6  1.0611e-7   3.99396e-6  …  3.76404e-8  1.94785e-7  8.70029e-8
 0.0         0.0         0.0            0.0         0.0         0.0
 0.0         0.0         0.0            0.0         0.0         0.0
 2.40126e-5  7.52929e-7  1.11037e-5     1.32678e-7  5.76731e-7  3.56244e-7
 3.10997e-5  3.46675e-7  1.91618e-6     1.95387e-7  1.03028e-6  5.21166e-7
 0.0         0.0         0.0         …  0.0         0.0         0.0
 0.0         0.0         0.0            0.0         0.0         0.0
 4.93689e-6  4.62439e-8  2.18639e-6     2.3285e-8   1.44899e-7  6.91399e-8
 ⋮        

## Reading in the data for Labor (L)

The L0 initial data in the .mat file read in the following cell contains the labor allocations for the first quarter in the year 2000. Since there is no dataset in the MAT file that directly gives the labor allocation data for 2007, we will use the mu values we read above to get our initial value of L for the dynamic programming algorithm.

In [10]:
using MAT
data = matread("Baseline_2000_2007_economy_actual.mat")

Dict{String, Any} with 23 entries:
  "T"                   => [0.392157; 0.179856; … ; 0.21978; 0.21978;;]
  "VARjn00"             => [676.845 307.099 … 10184.8 2583.86; 484.319 6.21808 …
  "Din00"               => [0.427487 0.0 … 0.000749794 0.000346362; 0.0 0.59431…
  "New_series_xbilat"   => [5272.99 0.0 … 3.54637 6.37772; 0.0 1065.06 … 1.4617…
  "Sn00"                => [0.0; -0.0; … ; 0.0; 0.0;;]
  "B"                   => [0.377534 0.434392 … 0.75094 0.390661; 0.377534 0.43…
  "io"                  => [0.00306927; 0.00063041; … ; 0.0151223; 0.00686046;;]
  "N"                   => 87.0
  "maxit"               => 1.0e20
  "series_Ljn0hat"      => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 ……
  "VALjn00"             => [1115.96 399.863 … 3377.93 4030.22; 798.53 8.09637 ……
  "New_series_wageshat" => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 ……
  "tol"                 => 1.0e-7
  "series_mu"           => [0.89191 0.000409915 … 0.0 0.0; 0.0151858 0.886235 ……
  "L0_ini

In [11]:
L0_initialization = data["L0_initial"]

1150×1 Matrix{Float64}:
 0.00535587895161499
 0.0001923349982624456
 0.00035622799678183625
 0.00030601899723542436
 1.7494799841951977e-5
 0.00010473999905377885
 0.00011064299900045116
 5.957529946179674e-5
 0.00035094699682954484
 0.00010079699908939992
 0.0001865469983147344
 0.00022409099797556192
 0.00021047399809857788
 ⋮
 3.513269968261119e-6
 6.962139937104028e-6
 0.00017056899845907967
 0.00012268499889166374
 6.521369941085943e-5
 3.0631999723270515e-5
 4.796219956670949e-5
 1.9104699827408143e-5
 0.00016904499847284748
 0.00015472699860219632
 7.4114899330446e-5
 0.00018808499830084012

In [12]:
L0_initialization_reshaped = reshape(L0_initialization, (n, j))

50×23 Matrix{Float64}:
 0.00535588   4.75829e-6   0.000997417  …  0.000645339  0.000439094
 0.000192335  4.90569e-5   0.000534768     0.000904484  0.000633204
 0.000356228  4.04634e-5   0.00322424      0.00108659   0.000192314
 0.000306019  4.66384e-5   0.00134835      0.000412448  0.00059435
 1.74948e-5   0.000145361  0.00190319      0.00197491   0.00425313
 0.00010474   5.66988e-5   0.0120632    …  0.00223952   0.000395697
 0.000110643  0.00042029   0.00563472      0.000909929  7.34995e-5
 5.95753e-5   0.00027968   0.00254162      0.00452679   0.000714667
 0.000350947  0.000192786  0.0035546       0.006005     6.09634e-6
 0.000100797  0.00186436   0.00420722      0.000195977  0.000121913
 0.000186547  0.00113946   0.001951     …  4.81196e-5   0.000201931
 0.000224091  0.000417134  0.00762362      0.000319757  7.76282e-5
 0.000210474  0.0003605    0.00910149      1.81817e-5   0.00061075
 ⋮                                      ⋱               
 0.00011543   0.000608747  0.00309253     

In [13]:
# people who stayed + people who came in - people who left
bigL= Array{Float64}(undef, n, j, 28)
bigL[:, :, 1] = L0_initialization_reshaped
for t in 2:28
    for a in 1:n
        for b in 1:j
            bigL[a, b, t] = (sum(bigL[c, d, t-1]*all_mus_reshaped[c, d, a, b, t] for c in 1:n, d in 1:j) - 
            sum(bigL[a, b, t-1]*all_mus_reshaped[a, b, c, d, t] for c in 1:n, d in 1:j if c!= a && d!= b))
        end
    end
end

In [14]:
bigL
# bigL is the variable that contains the labor allocations in all the sectors and locations from 2000 to 2007.
# This means that the 28th time period of bigL is the value of labor allocation(L) that should be used as the initial value for the dynamic algorithm.

50×23×28 Array{Float64, 3}:
[:, :, 1] =
 0.00535588   4.75829e-6   0.000997417  …  0.000645339  0.000439094
 0.000192335  4.90569e-5   0.000534768     0.000904484  0.000633204
 0.000356228  4.04634e-5   0.00322424      0.00108659   0.000192314
 0.000306019  4.66384e-5   0.00134835      0.000412448  0.00059435
 1.74948e-5   0.000145361  0.00190319      0.00197491   0.00425313
 0.00010474   5.66988e-5   0.0120632    …  0.00223952   0.000395697
 0.000110643  0.00042029   0.00563472      0.000909929  7.34995e-5
 5.95753e-5   0.00027968   0.00254162      0.00452679   0.000714667
 0.000350947  0.000192786  0.0035546       0.006005     6.09634e-6
 0.000100797  0.00186436   0.00420722      0.000195977  0.000121913
 0.000186547  0.00113946   0.001951     …  4.81196e-5   0.000201931
 0.000224091  0.000417134  0.00762362      0.000319757  7.76282e-5
 0.000210474  0.0003605    0.00910149      1.81817e-5   0.00061075
 ⋮                                      ⋱               
 0.00011543   0.000608747

## Reading in the data for total value added by all the labor and reading in the value of pi from one potential source.

In [15]:
using MAT
data = matread("Baseline_2000_2007_economy_actual.mat")
valaddlabour = data["VALjn00"] # divide this by L0 for wages.
pi0 = data["Din00"]

1914×87 Matrix{Float64}:
 0.427487     0.0          3.10145e-6   …  0.000749794  0.000346362
 0.0          0.594318     0.0             0.001017     0.000469817
 0.000705795  0.0          0.598479        0.000225427  0.000104125
 0.00120363   0.0          0.0             0.000831568  0.000384124
 0.000708008  0.000192805  0.00288905      0.000442682  0.000204493
 0.000437255  0.0          0.000473487  …  0.000665099  0.000307219
 7.04e-6      0.0          0.0             0.000317392  0.000146622
 0.0          0.0          0.0             0.00113895   0.000526123
 0.0139875    0.0          0.0             0.000265611  0.000122697
 0.0454015    0.0          0.0             0.000518305  0.000239414
 0.0          0.0          0.0          …  0.00053926   0.000249099
 0.0          0.0          0.000122629     0.00129924   0.000600164
 0.00225451   0.0          0.00104946      0.000413573  0.000191045
 ⋮                                      ⋱  ⋮            
 0.000170897  3.3804e-5    0.00071

As evident from Readme's in Caliendo's documents, out of the 87 columns in the 22x50 valaddlabor matrix, the first 50 are for US states so we only need to use them. It seems like the value added labor data is for 2007 and can be used as an initial value for the algorithm but this should definitely be verified.

In [19]:
valaddlabour=valaddlabour[:, 1:50]
valaddlabour

22×50 Matrix{Float64}:
  1115.96    399.863     1389.04    …   160.394    3431.08    109.519
   798.53      8.09637    140.257        77.528     396.7       6.58752
  3803.1      86.7559     812.954       372.933    6037.65     25.5878
   349.372   457.851       82.7391      565.37      126.241   374.627
  1545.86    104.739     1574.96       2897.25     2008.44    498.392
   610.638    11.9602     369.619   …   193.85     1459.29     20.1331
  1528.6      42.4435     975.928       179.022     683.463    54.7574
  2814.09     65.8686    1549.27        762.537    4786.18     86.5446
  1026.88      9.14278    449.569       105.582    5311.79     11.4312
  1177.43      2.80911   4565.84         81.8461   3529.28     24.5541
  2985.04     20.4016    3737.75    …   306.554    5171.11     21.0479
  1067.12      8.2592    1200.87        250.523    1159.26     21.3175
  9737.06   1831.02     12057.4        3476.48    13168.4    1365.92
  6111.41    912.938     8639.75       1703.84     8512.49

### Tasks still to be done

a) The value added matrix is only 22 X 50 and we don't know how to determine the value added by the non-employment sector. That value is needed so we can divide the value added data for all the sectors and states by all the initial labor allocations to get the initial wage for the dynamic algorithm for the dynamic algorithm

b) Find a source for pi that you are really confident in.

The pi values from a potential data source we read above as pi0 is a 1914 X 87 array. Just looking at the rows, the 1914 rows are blocks of 87 rows for each sector (22 here as data for non employment seems to be missing). The first 50 of these are US states, and the other 37 are countries. Looking at the 80 columns, the first 50 are US states, and the other 37 are countries. What the following chunk of code does is that it deletes the 37 rows of foreign countries from each row block, and the 37 columns for foreign countries. This algorithm does not seem like it will malfunction if we somehow find the pi values for the 23rd sector, because it seems to take all the rows block by block of 87 rows until it hits the end. But as always, cross check.

In [17]:
# Step 1: Keep only the first 50 columns
reduced_columns_array = pi0[:, 1:50]

# Step 2: Keep the first 50 rows out of every block of 87 rows
# Initialize an empty array to hold the processed blocks
processed_blocks = Array{typeof(reduced_columns_array[1,1]), 2}(undef, 0, 50)

# Process each block
for i in 0:((size(reduced_columns_array, 1) ÷ 87) - 1)
    block = reduced_columns_array[(i * 87 + 1):min((i + 1) * 87, end), :]
    processed_block = block[1:50, :]
    processed_blocks = vcat(processed_blocks, processed_block) # Concatenate the processed block
end

# `processed_blocks` is the final array with your requirements
processed_blocks

1100×50 Matrix{Float64}:
 0.427487     0.0          3.10145e-6   …  0.00900286  0.0
 0.0          0.594318     0.0             0.0         0.0
 0.000705795  0.0          0.598479        0.00850897  1.16536e-8
 0.00120363   0.0          0.0             0.00425895  0.0
 0.000708008  0.000192805  0.00288905      0.00688912  5.3804e-6
 0.000437255  0.0          0.000473487  …  0.00829615  0.000761952
 7.04e-6      0.0          0.0             0.00640348  0.0
 0.0          0.0          0.0             0.0         0.0
 0.0139875    0.0          0.0             0.0123672   3.36133e-7
 0.0454015    0.0          0.0             0.01661     0.0
 0.0          0.0          0.0          …  0.0         0.0
 0.0          0.0          0.000122629     0.0         6.61704e-5
 0.00225451   0.0          0.00104946      0.0700905   2.39794e-5
 ⋮                                      ⋱              
 0.0          0.0          0.0             0.0         0.0
 0.0          0.0          0.0             0.0     

# Setting up the variables for the dynamic programming algorithm along with the initial values as the first time period. Please find a good source for pi and the value added labor of non-employment sector. Random values are being used right now for pi and initial wages to keep the code for the algorithm intact.

In [16]:
μ = Array{Float64}(undef, n, j, i, k, nl+1)

50×23×50×23×201 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  

In [ ]:
μ[:, :, :, :, 1] = all_mus_reshaped[:, :, :, :, 28]

In [ ]:
for x in 1:n
    for y in 1:j
        matrix = rand(i, k)
        matrix2 = matrix/sum(matrix)
        π_0[x, y, :, :, 1] = matrix2
    end
end
π_0[:, :, :, :, 1]

In [ ]:
κdot = ones(n, j, i, k, nl+1)

In [ ]:
Ȧ = ones(n, j, nl+1)

In [ ]:
L = Array{Float64}(undef, n, j, nl+1)
L[:, :, 1] = bigL[:, :, 28]

In [ ]:
w = Array{Float64}(undef, n, j, nl+1)
w[:, :, 1] = rand(0.5:0.001:1.5, (n, j))

In [ ]:
using Distributions
θ_var = Frechet()
θ_list = rand(θ_var, (n, nl+1))

In [ ]:
u̇_0t = ones(n, j, nl+1)

In [ ]:
ẇ = ones(n, j, nl+1)

In [ ]:
ẇn = Array{Float64}(undef, n, j, nl+1)
Ṗ = Array{Float64}(undef, n, j, nl+1)
X = Array{Float64}(undef, n, j, nl+1)
α = Array{Float64}(undef, j, nl+1)
ẇ_old = Array{Float64}(undef, n, j, nl+1)
u̇_0t_old = Array{Float64}(undef, n, j, nl+1)
u̇_1t = Array{Float64}(undef, n, j, nl+1)

# The algorithm

In [ ]:
while !(all(all(0.95 .*u̇_0t[:, :, t] .<= u̇_0t_old[:, :, t] .<= 1.05.*u̇_0t[:, :, t]) for t in 1:nl+1) & all(0.95.*ones(n, j) .<= u̇_0t[:, :, nl+1] .<= 1.05.*ones(n, j)))
    println("OUTER LOOP STARTS")
    u̇_0t_old .= deepcopy(u̇_0t)
    for t in 2:nl
        for a in 1:n
            for b in 1:j
                for c in 1:i
                    for d in 1:k
                        denom = sum(μ[a, b, m, h, t-1]*((u̇_0t[m, h, t+1])^(β/ν)) for m in 1:n, h in 1:j)
                        numer = (μ[a, b, c, d, t-1]*((u̇_0t[c, d, t+1])^(β/ν)))  
                        # Please check the results of the above 2 lines and the line below by using print statements.
                        # Everything seems to be theoretically OK but when I used random data somehow the values generated
                        # here were complex numbers and I really couldn't pinpoint why.
                        μ[a, b, c, d, t] = numer/denom
                    end
                end
            end
        end
    end
    for t in 2:nl+1
        for a in 1:n
            for b in 1:j
                L[a, b, t] = sum(μ[c, d, a, b, t]*L[c, d, t-1] for c in 1:i, d in 1:k)
            end
        end
    end
    while !all(0.95*ẇ .<= ẇ_old .<= 1.05*ẇ)
        println("inner loop starts")
        ẇ_old .= deepcopy(ẇ)
        for t in 2:nl+1
            for a in 1:n
                for b in 1:j
                    Ṗ[n, j, t] = (sum(π_0[a, b, c, b, t]*((ẇ[c, b, t]*κdot[a, b, c, b, t])^(-θ_list[b, t]))*((Ȧ[c, b, t])^θ_list[b, t]) for c in 1:i))^(-1/θ_list[b, t])   
                end
            end
        end
        for t in 2:nl+1
            for a in 1:n
                for c in 1:i
                    for b in 1:j
                        π_0[a, b, c, b, t] = π_0[a, b, c, b]*(((ẇ[c, b]*κdot[a, b, c, b])/Ṗ[a, b])^(-θ_list[b]))*((Ȧ[c,b])^θ_list[b])
                    end
                end
            end  
        end
        for t in 1:nl+1
            for a in 1:n
                for b in 1:j
                    X[n, j, t] = w[n, j, t]*L[n, j, t]
                end
            end
        end
                    
        for t in 1:nl+1
            for b in 1:j
                α[b, t] = sum(X[k, b, t] for k in 1:n)/sum(X[j, l, t] for j in 1:n, l in 1:j)
            end
        end
        
        for t in 2:nl+1
            for a in 1:n
                for b in 1:j
                    w[a, b, t] = w[a, b, t-1]*ẇ[a, b, t]
                end
            end
        end
        
        for t in 1:nl+1
            for a in 1:n
                for b in 1:j
                    ẇn[a, b, t] = sum(π_0[c, b, a, b, t]*X[c, b, t] for c in 1:i)/(L[a, b, t]*w[a, b, t])
                end
            end
        end
        
        ẇ .= ẇn
        println("inner loop ends")
    end
    for t in 1:nl-1
        for a in 1:n
            for b in 1:j
                u̇_1t[a, b, t+1] = ẇ[a, b, t+1]*((sum(μ[a, b, c, d, t]*(u̇_0t[c, d, t+2])^(β/ν) for c in 1:i, d in 1:k))^(ν))
            end
        end
    end
    u̇_1t[:, :, nl+1] = ones(n, j)  # Discuss this with Professor Alder
    u̇_0t .= u̇_1t
    println("OUTER LOOP ENDS")
end